In [1]:
import os
import glob
#import tqdm
import numpy as np
#import pandas as pd
import multiprocessing
import pydicom as dicom
import nibabel as nib
#from keras import utils as kutils
from matplotlib import pyplot as plt

def load_label(label_fpath, transpose=False):
    #encode_name = label_fpath[-39: -7]
    label_data = nib.load(label_fpath)
    label_array = label_data.get_fdata()
    if transpose:
        label_array = np.transpose(label_array, axes=(2, 1, 0))
    return  label_array#, encode_name

In [3]:
import pandas as pd
info = pd.read_csv('/data2/pancreas/Nifti_data/data_list.csv')


In [4]:
id0_total = info.groupby("patient_id").size()[0]
num_of_patient = len(info.groupby("patient_id").size()) + id0_total - 1
print("num_of_patient :", num_of_patient)
print(len(info.groupby("patient_id").size()))
print(id0_total)

id0_healthy = info[info.type == "healthy"].groupby("patient_id").size()[0]
num_of_healthy_patient = len(info[info.type == "healthy"].groupby("patient_id").size()) + id0_healthy - 1
print(len(info[info.type == "healthy"].groupby("patient_id").size()))
print(id0_healthy)
print("num_of_healthy_patient :", num_of_healthy_patient)
print("num_of_tumor_patient :", num_of_patient - num_of_healthy_patient)

num_of_patient : 367
204
164
103
82
num_of_healthy_patient : 184
num_of_tumor_patient : 183


In [6]:
info['total'] = [1]*len(info)
pd.DataFrame(info.groupby(["resection", "200_list"]).sum().total)

total
resection 200_list         
0.0       test           14
          train          44
          validation     16
1.0       test            6
          train          16
          validation      4

In [39]:
from math import isnan
pd.DataFrame(info.groupby(["resection", "stage"]).sum().total)

total
resection stage       
0.0       I          1
          IIA        3
          IIB        4
          III       22
          IV        65
          X          1
1.0       IB         4
          II         1
          IIA       19
          IIB       44
          III       10
          IV         8

In [48]:
case_id1 = info.groupby("resection").get_group(1).case_id
case_id0 = info.groupby("resection").get_group(0).case_id
size1 = info.groupby("resection").get_group(1).size
size0 = info.groupby("resection").get_group(0).size
stage1 = info.groupby("resection").get_group(1).stage
stage0 = info.groupby("resection").get_group(0).stage

In [49]:
print(len(case_id1))
print(len(case_id0))

86
96


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Generator(nn.Module):
    def __init__(self, db='mnist', z_dim=128, cc_dim=1, dc_dim=10):
        super(Generator, self).__init__()
        self.db = db

        if self.db == 'mnist':
            self.fc = nn.Sequential(
                nn.Linear(z_dim + cc_dim + dc_dim, 1024),
                nn.BatchNorm2d(1024),
                nn.ReLU(),

                nn.Linear(1024, 128*7*7),
                nn.BatchNorm2d(128*7*7),
                nn.ReLU()
            )
            self.conv = nn.Sequential(
                # [-1, 128, 7, 7] -> [-1, 64, 14, 14]
                nn.ConvTranspose2d(128,64,4,2,1),
                nn.BatchNorm2d(64),
                nn.ReLU(),

                # -> [-1, 1, 28, 28]
                nn.ConvTranspose2d(64,1,4,2,1),
                nn.Tanh()
            )
        else:
            self.main = nn.Sequential(
                # [-1, z + cc + dc, 1, 1] -> [-1, 512, 4, 4]
                nn.ConvTranspose2d(z_dim + cc_dim + dc_dim, 1024, 4, 1, 0),

                nn.ConvTranspose2d(1024, 512, 4, 2, 1),
                nn.BatchNorm2d(512),
                nn.ReLU(),

                # [-1, 256, 8, 8]
                nn.ConvTranspose2d(512, 256, 4, 2, 1),
                nn.BatchNorm2d(256),
                nn.ReLU(),

                # [-1, 128, 16, 16]
                nn.ConvTranspose2d(256, 128, 4, 2, 1),
                nn.BatchNorm2d(128),
                nn.ReLU(),

                # [-1, 3, 32, 32]
                nn.ConvTranspose2d(128, 3, 4, 2, 1),
                nn.Tanh()
            )

    def forward(self, z):
        if self.db == 'mnist':
            # [-1, z]
            z = self.fc( z )

            # [-1, 128*7*7] -> [-1, 128, 7, 7]
            z = z.view(-1, 128, 7, 7)
            out = self.conv(z)
        else:
            # [-1, z] -> [-1, z, 1, 1]
            z = z.view(z.size(0), z.size(1), 1, 1)
            out = self.main( z )

        return out


class Discriminator(nn.Module):
    def __init__(self, db='mnist', cc_dim = 1, dc_dim = 10):
        super(Discriminator, self).__init__()
        self.db = db
        self.cc_dim = cc_dim
        self.dc_dim = dc_dim

        if self.db=='mnist':
            self.conv = nn.Sequential(
                # [-1, 1, 28, 28] -> [-1, 64, 14, 14]
                nn.Conv2d(1, 64, 4, 2, 1),
                nn.LeakyReLU(0.1, inplace=True),

                # [-1, 128, 7, 7]
                nn.Conv2d(64, 128, 4, 2, 1),
                nn.BatchNorm2d(128),
                nn.LeakyReLU(0.1, inplace=True),
            )
            self.fc = nn.Sequential(
                nn.Linear(128*7*7, 128),
                nn.BatchNorm2d(128),
                nn.LeakyReLU(0.1, inplace=True),
                nn.Linear(128, 1 + cc_dim + dc_dim)
            )
        else:
            self.main = nn.Sequential(
                # [-1, 3, 32, 32] -> [-1, 128, 16, 16]
                nn.Conv2d(3, 128, 4, 2, 1),
                nn.LeakyReLU(0.1, inplace=True),

                # [-1, 256, 8, 8]
                nn.Conv2d(128, 256, 4, 2, 1),
                nn.BatchNorm2d(256),
                nn.LeakyReLU(0.1, inplace=True),

                # [-1, 512, 4, 4]
                nn.Conv2d(256, 512, 4, 2, 1),
                nn.BatchNorm2d(512),
                nn.LeakyReLU(0.1, inplace=True),

                nn.Conv2d(512, 1024, 4, 2, 1),
                nn.BatchNorm2d(1024),
                nn.LeakyReLU(0.1, inplace=True),

                # [-1, 1 + cc_dim + dc_dim, 1, 1]
                nn.Conv2d(1024, 1 + cc_dim + dc_dim, 4, 1, 0)
            )

    def forward(self, x):
        if self.db == 'mnist':
            # -> [-1, 128*7*7]
            tmp = self.conv(x).view(-1, 128*7*7)

            # -> [-1, 1 + cc_dim + dc_dim]
            out = self.fc(tmp)
        else:
            # -> [-1, 1 + cc_dim + dc_dim]
            out = self.main(x).squeeze()

        # Discrimination Output
        out[:, 0] = F.sigmoid(out[:, 0].clone())

        # Continuous Code Output = Value Itself
        # Discrete Code Output (Class -> Softmax)
        out[:, self.cc_dim + 1:self.cc_dim + 1 + self.dc_dim] = F.softmax(out[:, self.cc_dim + 1:self.cc_dim + 1 + self.dc_dim].clone())

        return out

In [10]:
import numpy as np

from torch import nn
import torch.nn.functional as F

class Conv2d(nn.Conv2d):
    def reset_parameters(self):
        stdv = np.sqrt(6 / ((self.in_channels  + self.out_channels) * np.prod(self.kernel_size)))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.zero_()

class ConvTranspose2d(nn.ConvTranspose2d):
    def reset_parameters(self):
        stdv = np.sqrt(6 / ((self.in_channels  + self.out_channels) * np.prod(self.kernel_size)))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.zero_()

class Linear(nn.Linear):
    def reset_parameters(self):
        stdv = np.sqrt(6 / (self.in_features + self.out_features))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.zero_()

In [11]:
import os

import numpy as np

import torch
from torch.autograd import Variable
from torch import nn
import torch.nn.functional as F
import torch.optim as optim

from tqdm import tqdm_notebook



import pylab as plt

class InfoGAN:
    def __init__(self, gen, dis, embedding_len, z_len, c1_len, c2_len, c3_len):
        assert c1_len is not None and c1_len > 1, 'Must have a categorical code'

        self.gen = gen.cuda()
        self.dis = dis.cuda()

        self.embedding_len = embedding_len
        self.z_len = z_len
        self.c1_len = c1_len
        self.c2_len = c2_len
        self.c3_len = c3_len

        self.Q_cat = Linear(embedding_len, c1_len).cuda()
        self.qcat_optim = optim.Adam(self.Q_cat.parameters(), lr = 2e-4)
        if c2_len:
            self.Q_con = Linear(embedding_len, c2_len).cuda()
            self.qcon_optim = optim.Adam(self.Q_con.parameters(), lr = 2e-4)
        if c3_len:
            self.Q_bin = Linear(embedding_len, c3_len).cuda()
            self.qbin_optim = optim.Adam(self.Q_bin.parameters(), lr = 2e-4)

        self.g_optim = optim.Adam(self.gen.parameters(), lr = 1e-3)
        self.d_optim = optim.Adam(self.dis.parameters(), lr = 2e-4)


    def train_all(self, train_loader):
        nll = nn.NLLLoss().cuda()
        mse = nn.MSELoss().cuda()
        bce = nn.BCELoss().cuda()

        plt.figure(0, figsize = (32, 32))
        for epoch in range(100):
            pb = tqdm_notebook(total = train_loader.dataset.data_tensor.size()[0])
            pb.set_description('Epoch ' + str(epoch + 1))
            for i, (data, targets) in enumerate(train_loader):
                ones = Variable(torch.ones(data.size()[0], 1)).cuda()
                zeros = Variable(torch.zeros(data.size()[0], 1)).cuda()

                z_dict = self.get_z(data.size()[0])
                z = torch.cat([z_dict[k] for k in z_dict.keys()], dim = 1)

                data = Variable(data.float().cuda(async = True)) / 255
                targets = Variable(targets.float().cuda(async = True))

                # Forward pass on real MNIST
                out_dis, hid = self.dis(data)
                c1 = F.log_softmax(self.Q_cat(hid))
                loss_dis = mse(out_dis, ones) - torch.sum(targets * c1) / (torch.sum(targets) + 1e-3) # Loss for real MNIST

                # Forward pass on generated MNIST
                out_gen = self.gen(z)
                out_dis, hid = self.dis(out_gen)

                # Loss for generated MNIST
                loss_dis = loss_dis + mse(out_dis, zeros)
                loss_dis = loss_dis 

                # Zero gradient buffers for gen and Q_cat and backward pass
                self.dis.zero_grad()
                self.Q_cat.zero_grad()
                loss_dis.backward(retain_graph = True) # We need PyTorch to retain the graph buffers so we can run backward again later
                self.d_optim.step() # Apply the discriminator's update now since we have to delete its gradients later

                # And backward pass and loss for generator and update
                self.gen.zero_grad()
                loss_gen = mse(out_dis, ones)
                loss_gen.backward(retain_graph = True)
                self.dis.zero_grad() # Don't want the gradients of the generator's objective in the discriminator

                # Forward pass and loss for latent codes
                loss_q = 0

                c1 = F.log_softmax(self.Q_cat(hid))
                loss_q += nll(c1, torch.max(z_dict['cat'], dim = 1)[1])

                if self.c2_len:
                    c2 = self.Q_con(hid)
                    loss_q += 0.5 * mse(c2, z_dict['con']) # Multiply by 0.5 as we treat targets as Gaussian (and there's a coefficient of 0.5 when we take logs)
                    self.Q_con.zero_grad() # Zero gradient buffers before the backward pass
                if self.c3_len:
                    c3 = F.sigmoid(self.Q_bin(hid))
                    loss_q += bce(c3, z_dict['bin'])
                    self.Q_bin.zero_grad() # Zero gradient buffers before the backward pass

                # Backward pass for latent code objective
                loss_q.backward()

                # Do the updates for everything
                self.d_optim.step()
                self.g_optim.step()
                self.qcat_optim.step()

                if self.c2_len:
                    self.qcon_optim.step()
                if self.c3_len:
                    self.qbin_optim.step()



                pb.update(data.size()[0])
                pb.set_postfix(loss_dis = loss_dis.cpu().data.numpy()[0], loss_gen = loss_gen.cpu().data.numpy()[0], loss_q = loss_q.cpu().data.numpy()[0])

            pb.close()
            plt.subplot(10, 10, epoch + 1)
            plt.imshow(np.squeeze(np.transpose(out_gen.cpu().data.numpy()[0], (1, 2, 0))))


    # Generate a noise vector and random latent codes for generator
    def get_z(self, length, sequential = False):
        weights = torch.Tensor([0.1] * 10)

        z = {}
        if self.z_len:
            z['z'] = Variable(torch.randn(length, self.z_len)).cuda()

        if self.c1_len:
            if sequential:
                cat_noise = Variable(torch.arange(0, self.c1_len).repeat(length // self.c1_len).long()).cuda()
            else:
                cat_noise = Variable(torch.multinomial(weights, num_samples = length, replacement = True)).cuda().view(-1)
            onehot_noise = Variable(torch.zeros(length, self.c1_len)).cuda()
            onehot_noise.data.scatter_(1, cat_noise.data.view(-1, 1), 1)
            z['cat'] = onehot_noise

        if self.c2_len:
            #z['con'] = Variable(torch.randn(length, c2_len)).cuda()
            z['con'] = Variable(torch.rand(length, self.c2_len)).cuda() * 2 - 1

        if self.c3_len:
            z['bin'] = Variable(torch.bernoulli(0.5 * torch.ones(length, self.c3_len))).cuda().float()

        return z


    def run_dis(self, x):
        out = []
        out_dis, hid = self.dis(x)
        out += [out_dis]
        if self.c1_len:
            out += [F.softmax(self.Q_cat(hid))]
        if self.c2_len:
            out += [self.Q_con(hid)]
        if self.c3_len:
            out += [F.sigmoid(self.Q_bin(hid))]

        return out


    def save(self, directory):
        if not os.path.exists(directory):
            os.makedirs(directory)

        torch.save(self.gen.state_dict(), directory + 'gen.torch')
        torch.save(self.dis.state_dict(), directory + 'dis.torch')
        if self.c1_len:
            torch.save(self.Q_cat.state_dict(), directory + 'qcat.torch')
        if self.c2_len:
            torch.save(self.Q_con.state_dict(), directory + 'qcon.torch')
        if self.c3_len:
            torch.save(self.Q_bin.state_dict(), directory + 'qbin.torch')


    def load(self, directory):
        self.gen.load_state_dict(torch.load(directory + 'gen.torch'))
        self.dis.load_state_dict(torch.load(directory + 'dis.torch'))
        if self.c1_len:
            self.Q_cat.load_state_dict(torch.load(directory + 'qcat.torch'))
        if self.c2_len:
            self.Q_con.load_state_dict(torch.load(directory + 'qcon.torch'))
        if self.c3_len:
            self.Q_bin.load_state_dict(torch.load(directory + 'qbin.torch'))

In [3]:
!pip install torchvision --user

  Using cached https://files.pythonhosted.org/packages/2e/45/0f2f3062c92d9cf1d5d7eabd3cae88cea9affbd2b17fb1c043627838cb0a/torchvision-0.3.0-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/69/60/f685fb2cfb3088736bafbc9bdbb455327bdc8906b606da9c9a81bae1c81e/torch-1.1.0-cp36-cp36m-manylinux1_x86_64.whl


In [4]:
import torchvision

ImportError: /home/u/wartytw/.local/lib/python3.6/site-packages/torchvision/_C.cpython-36m-x86_64-linux-gnu.so: undefined symbol: _ZN3c1019UndefinedTensorImpl10_singletonE